In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import dash
import openpyxl
import os
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

##Iniciando o processo dados
navegador.get('https://statusinvest.com.br/acoes/eua/busca-avancada')
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[1]/div/div/div/button[2]').click()
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div[2]/a/span').click()
time.sleep(5)

#Fecha o navegador
navegador.quit()

## Renomeia o arquivo e move para pasta desejada
source = '/Users/scheiner/Downloads/statusinvest-busca-avancada.csv'
dest = '/Users/scheiner/Documents/GitHub/StatusInvest/international.csv'
os.rename(source, dest) 

print("Source path renamed to destination path successfully.")

Source path renamed to destination path successfully.


In [3]:
intl = pd.read_csv('international.csv', sep=';',  decimal=',', thousands='.', encoding="utf-8" )

intl['P/L_IDEAL'] = intl[' VPA'] / intl[' LPA']
intl['UP/DOWNSIDE'] = (intl[' VPA'] / intl['PRECO'])-1
    
value_intl = intl.loc[:, ['TICKER', 'PRECO', ' VPA' , 'DY', ' LPA', 'P/L', 'P/L_IDEAL', 'UP/DOWNSIDE', 'ROE', 
                           'P/VP', 'EV/EBIT' ,'LIQ. CORRENTE', 'MARG. LIQUIDA', 'DIV. LIQ. / PATRI.']]

value_intl.fillna(value=0, inplace=True)
value_intl.rename(columns={'LIQ. CORRENTE': "LIQ.CORR" }, inplace=True)
value_intl.rename(columns={'MARG. LIQUIDA': "MARG.LIQ" }, inplace=True)
value_intl.rename(columns={'DIV. LIQ. / PATRI.': "DIV.LIQ/PL" }, inplace=True)
    
    
value_intl = value_intl.loc[value_intl['DY'] >= 0].loc[
    value_intl['P/L'] >= 0.01].loc[value_intl['P/L'] < 10].loc[
    value_intl['ROE'] >= 6].loc[
    value_intl['EV/EBIT'] >= 0.01].loc[value_intl['EV/EBIT'] <= 10].loc[
    value_intl['LIQ.CORR'] >= 0.5]

value_intl

,TICKER,PRECO,VPA,DY,LPA,P/L,P/L_IDEAL,UP/DOWNSIDE,ROE,P/VP,EV/EBIT,LIQ.CORR,MARG.LIQ,DIV.LIQ/PL
11,AAP,59.59,45.79,7.97,6.37,9.35,7.188383,-0.231582,13.92,1.30,9.03,1.26,3.37,0.59
13,AAWW,102.48,107.04,0.00,12.43,8.25,8.611424,0.044496,11.61,0.96,8.47,0.96,7.82,0.51
23,ABG,233.57,149.14,0.00,45.49,5.13,3.278523,-0.361476,30.50,1.57,5.73,1.91,6.28,1.04
57,ACI,23.50,3.47,31.19,2.70,8.70,1.285185,-0.852340,77.76,6.76,9.05,0.78,1.86,4.09
158,AGCO,122.59,55.27,4.94,14.85,8.26,3.721886,-0.549148,26.87,2.22,7.59,1.57,7.84,0.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6104,YALA,5.11,2.50,0.00,0.53,9.65,4.716981,-0.510763,21.22,2.05,4.84,8.50,28.81,-1.01
6109,YGMZ,0.49,0.44,0.00,0.08,6.07,5.500000,-0.102041,18.50,1.12,5.17,1.57,6.33,0.56
6116,YPF,12.94,25.42,0.00,3.93,3.29,6.468193,0.964451,15.46,0.51,7.39,1.15,9.72,0.57
6147,ZIM,11.26,43.66,83.04,51.64,0.22,0.845469,2.877442,118.29,0.26,0.14,1.64,45.16,-0.04


In [4]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

ticker = value_intl['TICKER']
list_intl = ticker.to_list()

indicators_lis_intl = []
for intl in list_intl:

    #Criando url
    url = f"https://statusinvest.com.br/acoes/eua/{intl}"

    
    try:
        #Abrindo o navegador
        navegador.get(url)

        time.sleep(0.25)
    
        #Coletando infos
        NAME = navegador.find_element(By.XPATH, '//*[@id="main-header"]/div[2]/div/div[1]/div[1]/h1/small').text
        MIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[2]/div/div[2]/div/span[2]').text
        MAX_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[3]/div/div[2]/div/span[2]').text
        MIN_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[2]/div/div[1]/strong').text
        MAX_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[3]/div/div[1]/strong').text
        GAIN_12M = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[5]/div/div[1]/strong').text
        GAIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[5]/div/div[1]/strong').text
        ORIGEM = navegador.find_element(By.XPATH, '//*[@id="company-section"]/div[1]/div[3]/div/div[2]/div/div/div/strong/span').text
        SEGMENTO = navegador.find_element(By.XPATH, '//*[@id="company-section"]/div[1]/div[4]/div[2]/div[1]/div/div/div/a/strong').text
        
        

        #Criando dict
        dicionario = {  "intl": intl,
                        "NAME": NAME,
                        "MIN_MES": MIN_MES, 
                        "MAX_MES": MAX_MES, 
                        "MIN_52S": MIN_52S, 
                        "MAX_52S": MAX_52S, 
                        "VAL_12M": GAIN_12M,
                        "VAL_MES": GAIN_MES,
                        "ORIGEM": ORIGEM,
                        "SEGMENTO": SEGMENTO
                        
                        }

        #Criando lista de dicts
        indicators_lis_intl.append(dicionario)
    except Exception as e:
        print(e)

#Fecha o navegador
navegador.quit()

In [5]:
list_intl = pd.DataFrame.from_dict(indicators_lis_intl)

list_intl = list_intl.replace('-', '')
list_intl = list_intl.replace('-%', '')
list_intl['MIN_MES'] = (list_intl['MIN_MES']).str.lstrip('$')
list_intl['MAX_MES'] = (list_intl['MAX_MES']).str.lstrip('$')
list_intl['MIN_MES'] = (list_intl['MIN_MES']).str.strip('- ')
list_intl['MAX_MES'] = (list_intl['MAX_MES']).str.strip('- ')
list_intl['VAL_12M'] = (list_intl['VAL_12M']).str.rstrip('%')
list_intl['VAL_MES'] = (list_intl['VAL_MES']).str.rstrip('%')

list_intl.to_csv('list_intl.csv')

list_intl = pd.read_csv("list_intl.csv", sep=',', decimal=',', thousands='.', encoding="utf-8" )

list_intl.fillna(value=0, inplace=True)
list_intl = list_intl.drop(['Unnamed: 0'],axis=1)

value_intl.rename(columns={'TICKER': "TICKERS" }, inplace=True)

list_intl.rename(columns={'intl': "TICKERS" }, inplace=True)

invest_intl = pd.merge(value_intl, list_intl, on='TICKERS')

invest_intl = invest_intl[['TICKERS', 'NAME', 'PRECO', 'MIN_MES', 'MAX_MES', 'VAL_MES', 'MIN_52S', 'MAX_52S',  'VAL_12M', 
                       ' VPA', 'DY', ' LPA', 'P/L' ,'P/L_IDEAL', 'UP/DOWNSIDE', 'ROE', 'P/VP', 'EV/EBIT', 
                       'LIQ.CORR', 'MARG.LIQ', 'DIV.LIQ/PL', 'ORIGEM', 'SEGMENTO']]

invest_intl = invest_intl.loc[invest_intl['DY'] >= 0].loc[
    invest_intl['P/L'] >= 0.01].loc[invest_intl['P/L'] < 10].loc[
    invest_intl['ROE'] >= 6].loc[
    invest_intl['EV/EBIT'] >= 0.01].loc[invest_intl['EV/EBIT'] <= 10].loc[
    invest_intl['LIQ.CORR'] >= 0.5].loc[
    invest_intl['UP/DOWNSIDE'] >= 0.1]

#invest_intl[(invest_intl['TICKERS'] == 'MSFT')]
#invest_intl.loc[invest_intl['SEGMENTO'] == 'Technology'].loc[invest_intl['ORIGEM'] == 'NASDAQ']
# 
invest_intl


,TICKERS,NAME,PRECO,MIN_MES,MAX_MES,VAL_MES,MIN_52S,MAX_52S,VAL_12M,VPA,...,P/L_IDEAL,UP/DOWNSIDE,ROE,P/VP,EV/EBIT,LIQ.CORR,MARG.LIQ,DIV.LIQ/PL,ORIGEM,SEGMENTO
11,AMPY,AMPLIFY ENERGY CORP.,7.42,6.60,7.50,-0.13,5.66,10.21,-0.13,9.17,...,0.814387,0.235849,122.81,0.81,2.00,0.87,116.64,0.33,NYSE,Energy
35,BEST,BEST INC.,2.49,2.37,2.90,149.00,0.50,3.01,149.00,10.16,...,1.946360,3.080321,51.40,0.25,2.05,0.90,44.90,-1.61,NYSE,Transportation
37,BGFV,BIG 5 SPORTING GOODS CORPORATION,7.14,6.86,7.65,-38.97,6.86,13.56,-38.97,12.11,...,6.147208,0.696078,16.29,0.59,2.26,1.61,4.30,-0.11,NASDAQ,Consumer Services
39,BH,BIGLARI HOLDINGS INC.,176.03,176.03,186.76,43.70,115.60,212.62,43.70,198.63,...,5.660587,0.128387,17.67,0.89,3.62,1.26,29.10,-0.02,NYSE,Consumer Services
43,BRID,BRIDGFORD FOODS CORPORATION,11.05,10.61,11.67,-25.59,10.61,15.00,-25.59,14.04,...,2.830645,0.270588,35.35,0.79,1.41,4.18,17.39,-0.09,NASDAQ,Consumer Non-Durables
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,X,UNITED STATES STEEL CORPORATION,30.89,30.22,31.14,53.38,18.12,32.15,53.38,48.10,...,8.003328,0.557138,12.49,0.64,5.23,1.92,7.05,0.11,NYSE,Basic Industries
388,XCUR,EXICURE INC.,0.83,0.81,0.96,-55.14,0.71,1.93,-55.14,1.12,...,3.200000,0.349398,31.58,0.74,1.48,2.56,12.88,-0.24,NASDAQ,Health Care
390,XYF,X FINANCIAL,4.52,4.15,4.52,126.00,1.62,5.00,126.00,13.52,...,5.656904,1.991150,17.71,0.33,0.29,1.89,20.98,-0.25,NYSE,0
393,YPF,YPF SOCIEDAD ANONIMA,12.94,11.99,14.20,85.65,6.21,15.67,85.65,25.42,...,6.468193,0.964451,15.46,0.51,7.39,1.15,9.72,0.57,NYSE,Energy
